
# PCA paso a paso con Python (inspirado en StatQuest)

> Este notebook reproduce el ejemplo clásico de **PCA** explicado paso a paso,
incluyendo: centrado/estandarización, matriz de covarianza, **autovectores** / **autovalores**, **SVD**, **scores** (proyecciones), **varianza explicada**, **biplot**, reconstrucción y comparación con `scikit-learn`.

**Autor:** DataMining TA‑GPT · **Fecha:** 2025-09-02

**Requisitos:** `numpy`, `pandas`, `matplotlib`, `scikit-learn` (opcional `plotly`).


In [ ]:

# %% [markdown]
# ## 1. Importaciones y dataset sintético (2 genes × 6 muestras)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Datos simples (similar a "mice vs genes"): 6 muestras, 2 variables
# Cada fila es una muestra (M1..M6), columnas son "Gene1" y "Gene2"
X = np.array([
    [2.5, 2.4],
    [0.5, 0.7],
    [2.2, 2.9],
    [1.9, 2.2],
    [3.1, 3.0],
    [2.3, 2.7]
], dtype=float)

samples = [f"M{i}" for i in range(1, 7)]
df = pd.DataFrame(X, index=samples, columns=["Gene1", "Gene2"])
df


In [ ]:

# %% [markdown]
# ### Dispersión original
plt.figure()
plt.scatter(df["Gene1"], df["Gene2"])
for i, s in enumerate(samples):
    plt.annotate(s, (df.iloc[i,0], df.iloc[i,1]), xytext=(5,5), textcoords="offset points")
plt.xlabel("Gene1")
plt.ylabel("Gene2")
plt.title("Datos originales (sin centrar)")
plt.show()


In [ ]:

# %% [markdown]
# ## 2. Centrado y estandarización (opcional)
# Centrar: restar la media de cada columna
mu = df.mean(axis=0).values
X_centered = df.values - mu

# Estandarizar (z-score) — útil cuando las escalas difieren
sigma = df.std(axis=0, ddof=1).values
X_std = (df.values - mu) / sigma

pd.DataFrame(X_centered, index=samples, columns=df.columns).head()


In [ ]:

# Visualizar datos centrados
plt.figure()
plt.scatter(X_centered[:,0], X_centered[:,1])
for i, s in enumerate(samples):
    plt.annotate(s, (X_centered[i,0], X_centered[i,1]), xytext=(5,5), textcoords="offset points")
plt.axhline(0, linestyle="--"); plt.axvline(0, linestyle="--")
plt.xlabel("Gene1 (centrado)")
plt.ylabel("Gene2 (centrado)")
plt.title("Datos centrados en el origen")
plt.show()


In [ ]:

# %% [markdown]
# ## 3. Matriz de covarianza y autodescomposición (eigen)
# Usamos los datos centrados (recomendado para PCA clásico).
C = np.cov(X_centered, rowvar=False)  # 2x2
eigvals, eigvecs = np.linalg.eigh(C)  # eigh para matrices simétricas

# Ordenar por varianza (descendente)
idx = np.argsort(eigvals)[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

print("Covarianza:\n", C)
print("\nAutovalores (varianza por PC):", eigvals)
print("\nAutovectores (columnas = PCs):\n", eigvecs)


In [ ]:

# %% [markdown]
# ## 4. Proyección a PCs (scores) y varianza explicada
# Scores: proyección de los datos centrados sobre los autovectores
scores = X_centered @ eigvecs   # 6x2
explained_var = eigvals
explained_ratio = eigvals / eigvals.sum()

print("Scores (PC1, PC2):\n", np.round(scores, 3))
print("\nVarianza explicada:", np.round(explained_ratio, 3))

# Bar Scree plot
plt.figure()
plt.bar([1,2], explained_ratio, color="skyblue")
plt.xticks([1,2])
plt.ylabel("Proporción de varianza explicada")
plt.xlabel("Componente principal")
plt.title("Bar Scree plot (2D)")
plt.ylim(0,1)
plt.show()

# Proyección en PC1-PC2
plt.figure()
plt.scatter(scores[:,0], scores[:,1])
for i, s in enumerate(samples):
    plt.annotate(s, (scores[i,0], scores[i,1]), xytext=(5,5), textcoords="offset points")
plt.axhline(0, linestyle="--"); plt.axvline(0, linestyle="--")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Muestras en el espacio de PCs (scores)")
plt.show()


In [ ]:

# %% [markdown]
# ## 5. Biplot (scores + loadings)
loadings = eigvecs * np.sqrt(eigvals)  # escala típica para biplot
plt.figure()
plt.scatter(scores[:,0], scores[:,1], label="Muestras")
for i, s in enumerate(samples):
    plt.annotate(s, (scores[i,0], scores[i,1]), xytext=(5,5), textcoords="offset points")

# Vectores de variables (loadings)
for j, col in enumerate(df.columns):
    plt.arrow(0, 0, loadings[j,0], loadings[j,1], head_width=0.05, length_includes_head=True)
    plt.text(loadings[j,0]*1.1, loadings[j,1]*1.1, col)

plt.axhline(0, linestyle="--"); plt.axvline(0, linestyle="--")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Biplot (PC scores y loadings)")
plt.legend()
# plt.gca().set_aspect('equal', adjustable='box')
plt.show()


In [ ]:

# %% [markdown]
# ## 6. Reconstrucción desde k PCs
def reconstruct(scores, eigvecs, mu, k):
    # Usa solo las primeras k columnas (PCs)
    Vk = eigvecs[:, :k]
    Sk = scores[:, :k]
    X_rec = Sk @ Vk.T + mu  # Deshacer la proyección y sumar medias
    return X_rec

X_rec_1pc = reconstruct(scores, eigvecs, mu, k=1)
X_rec_2pc = reconstruct(scores, eigvecs, mu, k=2)

print("Reconstrucción con 1 PC:\n", np.round(X_rec_1pc, 3))
print("\nReconstrucción con 2 PCs (≈ original):\n", np.round(X_rec_2pc, 3))

# Comparar visualmente 1PC vs original
plt.figure()
plt.scatter(df["Gene1"], df["Gene2"], label="Original")
plt.scatter(X_rec_1pc[:,0], X_rec_1pc[:,1], marker="x", label="Reconstruido (1 PC)")
plt.xlabel("Gene1"); plt.ylabel("Gene2")
plt.title("Original vs Reconstrucción con 1 PC")
plt.legend(); plt.show()


In [ ]:

# %% [markdown]
# ## 7. SVD y su relación con PCA
# Para datos centrados: X_centered = U S V^T
U, S, Vt = np.linalg.svd(X_centered, full_matrices=False)
V = Vt.T
# En PCA, autovectores de la covarianza (eigvecs) coinciden con V
# y autovalores = (S^2)/(n-1)
eigvals_svd = (S**2) / (X_centered.shape[0]-1)
print("Autov. por SVD:", np.round(eigvals_svd, 6))
print("¿Coincide con eigen?:", np.allclose(eigvals_svd, eigvals))
print("¿Espacios V ~ eigvecs?:", np.allclose(np.abs(V), np.abs(eigvecs), atol=1e-6))


In [ ]:

# %% [markdown]
# ## 8. Comparación con `scikit-learn`
from sklearn.decomposition import PCA

pca = PCA(n_components=2)  # por defecto centra los datos
scores_skl = pca.fit_transform(df.values)
print("Explained variance ratio (sklearn):", np.round(pca.explained_variance_ratio_, 3))
print("Componentes (filas=PCs):\n", np.round(pca.components_, 6))

# Ajustes de signos pueden variar (PCs son únicos salvo signo)
sign_fix = np.sign(pca.components_[0,0])
components_skl = pca.components_.T * sign_fix
scores_skl_adj = scores_skl * sign_fix

print("\n¿Componentes ~ eigvecs?:", np.allclose(np.abs(components_skl), np.abs(eigvecs), atol=1e-6))



## 9. Consejos prácticos
- **Estandariza** si las variables están en escalas distintas.
- **Interpreta loadings** (dirección y contribución de variables) y **scores** (posición de muestras).
- El número de PCs puede guiarse por **scree plot**, ≥80% de varianza, o validación externa.
- PCA es lineal: patrones **no lineales** pueden requerir métodos como **t-SNE** o **UMAP**.
- Evita mezclar **variables categóricas crudas**; codifícalas (one-hot) o usa técnicas afines (MCA).



## Referencias sugeridas
- Shlens, J. (2014). *A Tutorial on Principal Component Analysis* (arXiv:1404.1100).
- Hastie, Tibshirani & Friedman (2009). *The Elements of Statistical Learning*, cap. 14.
- StatQuest: **Principal Component Analysis (PCA), Step-by-Step** (video).
